In [1]:
import torch
from torch import nn
import torch.nn.utils.prune as prune
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
from torchvision import models
from torchsummary import summary

c:\Users\user\anaconda3\envs\YOLOv8_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Check CUDA

In [2]:
if torch.cuda.is_available():
    cudnn.benchmark = True
    device = "cuda"
    print(torch.cuda.get_device_name())
else:
    device = "cpu"
    print("Use CPU")

Quadro RTX 3000 with Max-Q Design


## Load Model

In [3]:
PATH = r"my_weights\Resnet18_e50_b5_t70_v30.pth"
model = models.resnet18().to(device)
model.load_state_dict(torch.load(PATH))

<All keys matched successfully>

## Inspect Module

In [4]:
for name, layer in model.named_children():
    print(name, end=" / ")

conv1 / bn1 / relu / maxpool / layer1 / layer2 / layer3 / layer4 / avgpool / fc / 

In [5]:
module = model.conv1
print("## parameters: ", list(module.named_parameters()))
print("## buffers: ", list(module.named_buffers()))

## parameters:  [('weight', Parameter containing:
tensor([[[[-8.5304e-03, -2.8537e-03,  1.4496e-04,  ...,  5.6670e-02,
            1.7108e-02, -1.1546e-02],
          [ 1.2842e-02,  1.2794e-02, -1.0763e-01,  ..., -2.7050e-01,
           -1.2834e-01,  4.9514e-03],
          [-5.0904e-03,  6.1890e-02,  2.9732e-01,  ...,  5.2169e-01,
            2.5788e-01,  6.4414e-02],
          ...,
          [-2.7213e-02,  1.6944e-02,  7.4090e-02,  ..., -3.3203e-01,
           -4.2007e-01, -2.5796e-01],
          [ 2.9953e-02,  4.1496e-02,  6.3806e-02,  ...,  4.1398e-01,
            3.9341e-01,  1.6482e-01],
          [-1.4711e-02, -3.7172e-03, -2.3693e-02,  ..., -1.5023e-01,
           -8.1575e-02, -5.5166e-03]],

         [[-1.1180e-02, -2.5247e-02, -3.4059e-02,  ...,  3.0147e-02,
           -1.4401e-03, -2.6802e-02],
          [ 4.5294e-02,  3.4746e-02, -1.0429e-01,  ..., -3.1471e-01,
           -1.6252e-01, -2.5516e-03],
          [-9.5681e-04,  9.9350e-02,  4.0258e-01,  ...,  7.0756e-01,
        

## Pruning a Module

In [6]:
prune.random_unstructured(module, name="weight", amount=0.3)

Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

In [7]:
print("## parameters: ", list(module.named_parameters()))

## parameters:  [('weight_orig', Parameter containing:
tensor([[[[-8.5304e-03, -2.8537e-03,  1.4496e-04,  ...,  5.6670e-02,
            1.7108e-02, -1.1546e-02],
          [ 1.2842e-02,  1.2794e-02, -1.0763e-01,  ..., -2.7050e-01,
           -1.2834e-01,  4.9514e-03],
          [-5.0904e-03,  6.1890e-02,  2.9732e-01,  ...,  5.2169e-01,
            2.5788e-01,  6.4414e-02],
          ...,
          [-2.7213e-02,  1.6944e-02,  7.4090e-02,  ..., -3.3203e-01,
           -4.2007e-01, -2.5796e-01],
          [ 2.9953e-02,  4.1496e-02,  6.3806e-02,  ...,  4.1398e-01,
            3.9341e-01,  1.6482e-01],
          [-1.4711e-02, -3.7172e-03, -2.3693e-02,  ..., -1.5023e-01,
           -8.1575e-02, -5.5166e-03]],

         [[-1.1180e-02, -2.5247e-02, -3.4059e-02,  ...,  3.0147e-02,
           -1.4401e-03, -2.6802e-02],
          [ 4.5294e-02,  3.4746e-02, -1.0429e-01,  ..., -3.1471e-01,
           -1.6252e-01, -2.5516e-03],
          [-9.5681e-04,  9.9350e-02,  4.0258e-01,  ...,  7.0756e-01,
   

In [8]:
print("## buffers: ", list(module.named_buffers()))

## buffers:  [('weight_mask', tensor([[[[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 0., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [0., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 0., 1., 1.],
          [1., 1., 0.,  ..., 1., 1., 0.]],

         [[1., 0., 0.,  ..., 0., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 0.,  ..., 1., 1., 1.],
          [1., 0., 1.,  ..., 1., 1., 1.],
          [0., 1., 1.,  ..., 0., 1., 1.]],

         [[1., 1., 1.,  ..., 0., 1., 1.],
          [0., 1., 1.,  ..., 1., 1., 1.],
          [1., 0., 1.,  ..., 1., 1., 0.],
          ...,
          [0., 0., 1.,  ..., 0., 1., 0.],
          [0., 1., 1.,  ..., 1., 1., 0.],
          [0., 1., 1.,  ..., 1., 1., 1.]]],


        [[[1., 1., 1.,  ..., 1., 0., 0.],
          [0., 0., 1.,  ..., 1., 0., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 0., 1

In [9]:
print(module.weight)

tensor([[[[-8.5304e-03, -2.8537e-03,  1.4496e-04,  ...,  5.6670e-02,
            1.7108e-02, -1.1546e-02],
          [ 1.2842e-02,  0.0000e+00, -1.0763e-01,  ..., -2.7050e-01,
           -1.2834e-01,  4.9514e-03],
          [-5.0904e-03,  6.1890e-02,  2.9732e-01,  ...,  5.2169e-01,
            2.5788e-01,  6.4414e-02],
          ...,
          [-0.0000e+00,  1.6944e-02,  7.4090e-02,  ..., -3.3203e-01,
           -4.2007e-01, -2.5796e-01],
          [ 2.9953e-02,  4.1496e-02,  6.3806e-02,  ...,  0.0000e+00,
            3.9341e-01,  1.6482e-01],
          [-1.4711e-02, -3.7172e-03, -0.0000e+00,  ..., -1.5023e-01,
           -8.1575e-02, -0.0000e+00]],

         [[-1.1180e-02, -0.0000e+00, -0.0000e+00,  ...,  0.0000e+00,
           -1.4401e-03, -2.6802e-02],
          [ 4.5294e-02,  3.4746e-02, -1.0429e-01,  ..., -3.1471e-01,
           -1.6252e-01, -2.5516e-03],
          [-9.5681e-04,  9.9350e-02,  4.0258e-01,  ...,  7.0756e-01,
            3.6813e-01,  1.2365e-01],
          ...,
     

In [10]:
print(module._forward_pre_hooks)

OrderedDict([(0, <torch.nn.utils.prune.RandomUnstructured object at 0x000001AD05E66790>)])


In [20]:
model_path = f'my_weights/Resnet18_e50_b5_t70_v30(conv1 sp=0.3).pth'
torch.save(model.state_dict(), model_path)